## 지니차트 Top100

In [91]:
import requests
import pandas as pd
from tqdm import tqdm
from datetime import datetime as dt
from bs4 import BeautifulSoup

In [75]:
url = 'https://www.genie.co.kr/chart/top200'
res = requests.get(url)
res.text

'<br>\r\n<br>\r\n<center>\r\n<br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [76]:
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
res = requests.get(url, headers=header)
res.text[:500]

'\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n\t\n\n\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n<!--[if lte IE 6]><html dir="ltr" class="no-js ie6"><![endif]-->\n<!--[if lte IE 7]><html dir="ltr" class="no-js ie7"><![endif]-->\n<!--[if IE 8]><html dir="ltr" class="no-js ie8"><![endif]-->\n<!--[if IE 9]><html dir="ltr" class="no-js ie9"><![endif]-->\n<!--[if (gte IE 10)|!(IE)]><!--><html dir="ltr" class="no-js"><!--<![endif]-->\n<title>음악, 그리고 설레임 - 지니</title>\n<meta charset="utf-8'

In [77]:
soup = BeautifulSoup(res.text, 'html.parser')

- 원하는 데이터 찾기

In [92]:
now=res.now()
date_str = now.strftime('%Y%m%d')
hour = now.strftime('%H')
date_str

AttributeError: 'Response' object has no attribute 'now'

In [79]:
gini = soup.select('.list')
len(gini)

148

In [80]:
trs = soup.select('#body-content > div.newest-list > div > table > tbody > tr')

- 하나를 선택해서 원하는 정보 추출

In [81]:
tr = trs[0]
rank = tr.select_one('td.number').text[0:2].strip()

In [82]:
title = tr.select_one('.albumtitle.ellipsis').get_text().strip()
title

'Love Lee'

In [83]:
artist = tr.select_one('.artist.ellipsis').get_text().strip()
artist

'AKMU (악뮤)'

In [84]:
album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
album

'Love Lee'

In [85]:
img = tr.select_one('tr > td:nth-child(3) > a > img')['src']
img

'//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/181/610/84181610_1692585955705_1_140x140.JPG/dims/resize/Q_80,0'

In [86]:
data = []
for page in tqdm(range(1,5)):
    url = f'{url}?page={page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    gini = soup.select('.list')

for tr in gini:
    date_str = soup.select_one('.date').get_text().strip()
    gini = soup.select('.list')
    trs = soup.select('#body-content > div.newest-list > div > table > tbody > tr')
    rank = tr.select_one('td.number').text[0:2].strip()
    title = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    img = tr.select_one('tr > td:nth-child(3) > a > img')['src']
    data.append({'순위':rank, '제목':title, '아티스트':artist, '앨범':album, '이미지':img})

100%|██████████| 4/4 [00:00<00:00, 29.30it/s]


In [87]:
gini = pd.DataFrame(data)
gini

""
